In [1]:
# 🛠️ Installation (nur beim ersten Mal nötig)
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-deu
!apt-get install -y poppler-utils
!pip install pytesseract pdf2image pymupdf transformers sentencepiece

# 📦 Imports
import os
import pytesseract
from pdf2image import convert_from_path
import fitz  # PyMuPDF
from transformers import MarianMTModel, MarianTokenizer
from google.colab import drive

# 🔧 Tesseract OCR Settings
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"
ocr_language = "deu"

# 📂 Drive mounten
drive.mount('/content/drive')

# 🔹 Drive-Ordner (ggf. korrigieren)
drive_folder = "/content/drive/MyDrive/data/"
if not os.path.exists(drive_folder):
    drive_folder = "/content/drive/My Drive/data/"



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-deu
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 744 kB of archives.
After this operation, 1,540 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-deu all 1:4.00~git30-7274cfa-1.1 [744 kB]
Fetched 744 kB in 0s (2,297 kB/s)
Selecting previously unselected package tesseract-ocr-deu.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-deu_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-deu (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-

In [5]:
# PDF-Datei
pdf_path = drive_folder + "KarlHomann/MinnameierHappyVictimizer.pdf"

# Funktion: Direkttext aus PDF extrahieren
def extract_text_direct(pdf_path):
    doc = fitz.open(pdf_path)
    text = "".join([seite.get_text() for seite in doc])
    doc.close()
    return text.strip()

# OCR-Fallback
def extract_text_ocr(pdf_path):
    images = convert_from_path(pdf_path)
    ocr_text = ""
    for i, img in enumerate(images):
        print(f" OCR Seite {i+1}")
        ocr_text += pytesseract.image_to_string(img, lang=ocr_language) + "\n"
    return ocr_text.strip()

#  Translator vorbereiten: Englisch → Deutsch
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text, max_chunk_len=512):
    sentences = text.split('. ')
    translated = []
    buffer = ""
    for sentence in sentences:
        if len(buffer + sentence) < max_chunk_len:
            buffer += sentence + ". "
        else:
            tokens = tokenizer.prepare_seq2seq_batch([buffer], return_tensors="pt")
            translated.append(model.generate(**tokens))
            buffer = sentence + ". "
    if buffer:
        tokens = tokenizer.prepare_seq2seq_batch([buffer], return_tensors="pt")
        translated.append(model.generate(**tokens))

    output = []
    for t in translated:
        output.extend(tokenizer.batch_decode(t, skip_special_tokens=True))
    return " ".join(output)

#  Text extrahieren
print(" Versuche, Text direkt zu extrahieren...")
text = extract_text_direct(pdf_path)
if not text:
    print(" Kein Text gefunden – OCR wird gestartet...")
    text = extract_text_ocr(pdf_path)
else:
    print(" Text direkt gefunden.")

#  Bereinigen
clean_text = text.replace("\n", " ").replace("  ", " ").strip()

#  Übersetzen
#print(" Übersetze ins Deutsche...")
#translated_text = translate_text(clean_text)

#  Speichern
output_path = pdf_path.replace(".pdf", ".txt") #"_deutsch.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(clean_text) # f.write(translated_text)

print(f" (Übersetzter) Text gespeichert unter:\n{output_path}")


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


 Versuche, Text direkt zu extrahieren...
 Kein Text gefunden – OCR wird gestartet...
 OCR Seite 1
 OCR Seite 2
 OCR Seite 3
 OCR Seite 4
 OCR Seite 5
 OCR Seite 6
 OCR Seite 7
 OCR Seite 8
 OCR Seite 9
 OCR Seite 10
 OCR Seite 11
 OCR Seite 12
 OCR Seite 13
 OCR Seite 14
 OCR Seite 15
 OCR Seite 16
 OCR Seite 17
 (Übersetzter) Text gespeichert unter:
/content/drive/MyDrive/data/KarlHomann/MinnameierHappyVictimizer.txt
